In [1]:
import os
os.chdir('..')

In [2]:
from src.utils import *
from src.mol.preprocess import *
from src.mol.features import *
from src.mol.mpl_rdkit_utils import plot_mol_graph
from src.mol.mol_dataset import MolDataset
from src.mol.gnn import GNN

In [3]:
all_candidates_train = load_pkl('cache/pkl/v1/candidates.train.pkl')
all_candidates_test = load_pkl('cache/pkl/v1/candidates.test.pkl')
mapped_smiles = mapped_smiles_reader('cache/mapped_drugs/all_mapped.txt')
x_train, y_train = candidate_smiles(all_candidates_train, mapped_smiles)
x_test, y_test = candidate_smiles(all_candidates_test, mapped_smiles)
dataset_train_mol1 = MolDataset(x_train, element=1)
# dataset_train_mol2 = MolDataset(x_train, element=2)
# dataset_test_mol1 = MolDataset(x_test, element=1)
# dataset_test_mol2 = MolDataset(x_test, element=2)

Converting SMILES to PyG: 100%|██████████| 27792/27792 [00:19<00:00, 1440.04it/s]


# GCN

In [4]:
model = GNN()
print(model)

GNN(
  (atom_encoder): Embedding(119, 64, padding_idx=0)
  (bond_encoder): Embedding(22, 16, padding_idx=0)
  (boolean_encoder): Embedding(3, 2, padding_idx=2)
  (conv1): GATv2Conv(74, 512, heads=1)
  (conv2): GATv2Conv(512, 512, heads=1)
  (conv3): GATv2Conv(512, 512, heads=1)
  (conv4): GATv2Conv(512, 512, heads=1)
)


In [5]:
for mol in dataset_train_mol1:
    try:
        model(mol)
    except:
        print(mol.x)